<a href="https://colab.research.google.com/github/sanjanascorner/neural-architecture-search/blob/master/NanoNAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from multiprocessing import Process,Queue
from pathlib import Path
import tensorflow as tf
import numpy as np
import subprocess
import datetime
import os
import re

In [2]:
!chmod +x stm32tflm


In [10]:
class NanoNAS :
    def __init__(self, max_ram, max_flash, max_macc, path_to_training_set, val_split, cache=False, input_shape=(50,50,3), save_path='./') :
        self.path_to_training_set = path_to_training_set
        self.num_classes = len(next(os.walk(path_to_training_set))[1])
        self.val_split = val_split
        self.input_shape = input_shape
        self.max_ram = max_ram
        self.max_flash = max_flash
        self.max_macc = max_macc
        self.cache = cache
        self.save_path = Path(save_path)
        os.makedirs(self.save_path,exist_ok=True)
        self.path_to_resulting_model = self.save_path / 'resulting_architecture.h5'
        self.path_to_quantized_resulting_model = self.save_path / 'resulting_architecture.tflite'

    # k number of kernels of the first convolutional layer
    # c number of cells added upon the first convolutional layer
    # pre-processing pipeline not included in MACC computation
    def Model(self, k, c) :
        kernel_size = (3,3)
        pool_size = (2,2)
        pool_strides = (2,2)

        number_of_cells_limited = False
        macc = 0

        inputs = tf.keras.Input(shape=self.input_shape)

        #convolutional base
        n = k
        multiplier = 2

        #first convolutional layer
        c_in = self.input_shape[2]
        x = tf.keras.layers.Conv2D(n, kernel_size, padding='same')(inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        macc = macc + (c_in * kernel_size[0] * kernel_size[1] * x.shape[1] * x.shape[2] * x.shape[3])

        #adding cells
        for i in range(1, c + 1) :
            if x.shape[1] <= 1 or x.shape[2] <= 1 :
                number_of_cells_limited = True
                break;
            n = int(np.ceil(n * multiplier))
            multiplier = multiplier - 2**-i
            x = tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=pool_strides, padding='valid')(x)
            x = tf.keras.layers.Conv2D(n, kernel_size, padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation('relu')(x)
            c_in = x.shape[3]
            macc = macc + (c_in * kernel_size[0] * kernel_size[1] * x.shape[1] * x.shape[2] * x.shape[3])

        #classifier
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        outputs = tf.keras.layers.Dense(self.num_classes, activation='softmax')(x)
        macc = macc + (x.shape[1] * outputs.shape[1])

        model = tf.keras.Model(inputs=inputs, outputs=outputs)

        return model, macc, number_of_cells_limited

    def load_training_set(self, batch_size=1):
        if 3 == self.input_shape[2] :
            color_mode = 'rgb'
        elif 1 == self.input_shape[2] :
            color_mode = 'grayscale'

        train_ds = tf.keras.utils.image_dataset_from_directory(
            directory= self.path_to_training_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=batch_size,
            image_size=self.input_shape[0:2],
            shuffle=True,
            seed=11,
            validation_split=self.val_split,
            subset='training'
        )

        validation_ds = tf.keras.utils.image_dataset_from_directory(
            directory= self.path_to_training_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=batch_size,
            image_size=self.input_shape[0:2],
            shuffle=True,
            seed=11,
            validation_split=self.val_split,
            subset='validation'
        )

        if self.cache :
            train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
            validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        else :
            train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
            validation_ds = validation_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

        return train_ds, validation_ds

    def compile_model(self, model, learning_rate):
         opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

         model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    def quantize_model(self, model, train_ds, path_to_tflite_model) :
        def representative_dataset():
            for data in train_ds.rebatch(1).take(150) :
                yield [tf.dtypes.cast(data[0], tf.float32)]

        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8  # or tf.int8
        converter.inference_output_type = tf.uint8  # or tf.int8
        tflite_quant_model = converter.convert()

        with open(path_to_tflite_model, 'wb') as f:
            f.write(tflite_quant_model)

    def evaluate_flash_and_peak_ram_occupancy(self, model, train_ds) :
        #it must be done after one epoch of training, at least
        path_to_tflite_model = self.save_path / 'temp.tflite'
        #quantize model to evaluate its peak RAM occupancy and its Flash occupancy
        self.quantize_model(model, train_ds, path_to_tflite_model)

        #evaluate its peak RAM occupancy and its Flash occupancy using STMicroelectronics' script named "stm32tflm"
        #found inside the linux package of X-CUBE-AI at the following path:
        #"path/to/en.x-cube-ai-linux_v8.0.1/stm32ai-linux-8.0.1/linux/stm32tflm".
        #The package can be downloaded at https://www.st.com/en/embedded-software/x-cube-ai.html#get-software.
        proc = subprocess.Popen(["./stm32tflm", path_to_tflite_model], stdout=subprocess.PIPE)
        try:
            outs, errs = proc.communicate(timeout=15)
            flash, ram = re.findall(r'\d+', str(outs))
            os.remove(path_to_tflite_model)
        except subprocess.TimeoutExpired:
            proc.kill()
            outs, errs = proc.communicate()
            os.remove(path_to_tflite_model)
            print("stm32tflm error")
            exit()

        return int(flash), int(ram)

    def evaluate_model_process(self, q, k, c) :
        epochs = 3
        search_learning_rate = 0.001
        search_batch_size = 16

        train_ds, validation_ds = self.load_training_set(search_batch_size)

        model, macc, number_of_cells_limited = self.Model(k, c)
        self.compile_model(model, search_learning_rate)
        hist = model.fit(train_ds, epochs=epochs, validation_data=validation_ds, validation_freq=1)
        flash, ram = self.evaluate_flash_and_peak_ram_occupancy(model, train_ds)
        feasible = macc <= self.max_macc and flash <= self.max_flash and ram <= self.max_ram and not number_of_cells_limited
        if self.save_search_history and feasible :
            model_name = 'k' + str(k) + '_c' + str(c)
            new_dir = self.save_path / 'search_history' / f"search_learning_rate_{search_learning_rate}" / f"search_batch_size{search_batch_size}" / model_name
            os.makedirs(new_dir)
            np.save(new_dir / f"{model_name}_hist.npy", hist.history)
        q.put({'k': k,
               'c': c if not number_of_cells_limited else f"{c} (Not feasible)",
               'RAM': ram if ram <= self.max_ram else f"{ram} (Outside the upper bound of {ram - self.max_ram} Byte)",
               'Flash': flash if flash <= self.max_flash else f"{flash} (Outside the upper bound of {flash - self.max_flash} Byte)",
               'MACC': macc if macc <= self.max_macc else f"{macc} (Outside the upper bound of {macc - self.max_macc} MAC)",
               'max_val_acc': np.around(np.amax(hist.history['val_accuracy']), decimals=3)
               if feasible else -3})

    def search(self, save_search_history=False) :
        self.save_search_history = save_search_history

        start = datetime.datetime.now()

        best_architecture = {'k': -1, 'c': -1, 'max_val_acc': -2}
        new_architecture = {'k': -1, 'c': -1, 'max_val_acc': -1}

        k = 1
        while(new_architecture['max_val_acc'] > best_architecture['max_val_acc']) :
            best_architecture = new_architecture
            c = -1
            previous_architecture = {'k': -1, 'c': -1, 'max_val_acc': -2}
            current_architecture = {'k': -1, 'c': -1, 'max_val_acc': -1}
            while(current_architecture['max_val_acc'] > previous_architecture['max_val_acc']) :
                previous_architecture = current_architecture
                c = c + 1
                q = Queue()
                p = Process(target=self.evaluate_model_process, args=(q, k, c,))
                p.start()
                p.join()
                if q.empty() :
                    #the machine was not able to train the architecture for one epoch
                    current_architecture = {'k': k, 'c': c, 'max_val_acc': -1}
                else :
                    current_architecture = q.get()
                print(f"\n\n{current_architecture}\n\n")
            new_architecture = previous_architecture
            k = k + 1

        end = datetime.datetime.now()

        if 0 < best_architecture['max_val_acc'] :
            print(f"Resulting architecture: {best_architecture}\n")
            print(f"Elapsed time (search): {end-start}\n")

            self.resulting_architecture = best_architecture
        else :
            print("No feasible solution found.\n")
            exit(0)

    def train_process(self, training_epochs, training_learning_rate, training_batch_size) :
        train_ds, validation_ds = self.load_training_set(training_batch_size)
        model = self.Model(self.resulting_architecture['k'], self.resulting_architecture['c'])[0]
        self.compile_model(model, training_learning_rate)
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.path_to_resulting_model),
            save_weights_only=False, monitor='val_accuracy',
            mode='auto', save_best_only=True, verbose=1)

        hist = model.fit(train_ds, epochs=training_epochs, validation_data=validation_ds, validation_freq=1, callbacks=[model_checkpoint_callback])

        print('\nmax val acc: ' + str(round(np.amax(hist.history['val_accuracy']), 3)))

        print(f"\nKeras model saved in: {self.path_to_resulting_model}\n")

    def train(self, training_epochs, training_learning_rate, training_batch_size) :
        start = datetime.datetime.now()
        p = Process(target=self.train_process, args=((training_epochs, training_learning_rate, training_batch_size,)))
        p.start()
        p.join()
        end = datetime.datetime.now()
        print(f"Elapsed time (training): {end-start}\n")

    def apply_uint8_post_training_quantization_process(self) :
        train_ds, validation_ds = self.load_training_set()

        model = tf.keras.models.load_model(self.path_to_resulting_model)

        self.quantize_model(model, train_ds, self.path_to_quantized_resulting_model)

        print(f"\nTflite model saved in: {self.path_to_quantized_resulting_model.resolve()}\n")

    def apply_uint8_post_training_quantization(self) :
        p = Process(target=self.apply_uint8_post_training_quantization_process)
        p.start()
        p.join()

    def load_test_set(self, path_to_test_set, batch_size=1):
        if 3 == self.input_shape[2] :
            color_mode = 'rgb'
        elif 1 == self.input_shape[2] :
            color_mode = 'grayscale'

        test_ds = tf.keras.utils.image_dataset_from_directory(
            directory= path_to_test_set,
            labels='inferred',
            label_mode='categorical',
            color_mode=color_mode,
            batch_size=batch_size,
            image_size=self.input_shape[0:2]
        )

        if self.cache :
            test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        else :
            test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

        return test_ds

    def test_keras_model_process(self, path_to_test_set) :
        test_ds = self.load_test_set(path_to_test_set)
        model = tf.keras.models.load_model(self.path_to_resulting_model)

        #evaluate keras model
        print(f"\nKeras model test accuracy: {model.evaluate(test_ds)[1]}\n")

    def test_keras_model(self, path_to_test_set) :
        p = Process(target=self.test_keras_model_process, args=(path_to_test_set,))
        p.start()
        p.join()

    def test_tflite_model_process(self, path_to_test_set) :
        test_ds = self.load_test_set(path_to_test_set)
        interpreter = tf.lite.Interpreter(f"{self.path_to_quantized_resulting_model.resolve()}")
        interpreter.allocate_tensors()

        output = interpreter.get_output_details()[0]  # Model has single output.
        input = interpreter.get_input_details()[0]  # Model has single input.

        correct = 0
        wrong = 0

        for image, label in test_ds.rebatch(1) :
            # Check if the input type is quantized, then rescale input data to uint8
            if input['dtype'] == tf.uint8:
                input_scale, input_zero_point = input["quantization"]
                image = image / input_scale + input_zero_point
            input_data = tf.dtypes.cast(image, tf.uint8)
            interpreter.set_tensor(input['index'], input_data)
            interpreter.invoke()
            if label.numpy().argmax() == interpreter.get_tensor(output['index']).argmax() :
                correct = correct + 1
            else :
                wrong = wrong + 1
        print(f"\nTflite model test accuracy: {correct/(correct+wrong)}")

        print(f"\nTflite model in: {self.path_to_quantized_resulting_model.resolve()}\n")

    def test_tflite_model(self, path_to_test_set) :
        p = Process(target=self.test_tflite_model_process, args=(path_to_test_set,))
        p.start()
        p.join()

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
!unzip melanoma-skin-cancer-dataset-of-10000-images.zip


In [11]:
import numpy as np
import tensorflow as tf

input_shape = (50,50,3)

#Each dataset must comply with the following structure
#main_directory/
#...class_a/
#......a_image_1.jpg
#......a_image_2.jpg
#...class_b/
#......b_image_1.jpg
#......b_image_2.jpg
path_to_training_set = r"/content/melanoma_cancer_dataset/train"
val_split = 0.3
path_to_test_set=r"/content/melanoma_cancer_dataset/test"

#whether or not to cache datasets in memory
#if the dataset cannot fit in the main memory, the application will crash
cache = True

#target: STM32L010RBT6
#75 CoreMark, 20 kiB RAM, 128 kiB Flash
ram_upper_bound = 20480
flash_upper_bound = 131072
MACC_upper_bound = 750000 #CoreMark * 1e4

nanoNAS = NanoNAS(ram_upper_bound, flash_upper_bound, MACC_upper_bound, path_to_training_set, val_split, cache, input_shape, save_path='./results')

#search
nanoNAS.search(save_search_history=False)

#train resulting architecture
nanoNAS.train(training_epochs=100, training_learning_rate=0.01, training_batch_size=128)

#apply uint8 post trainig quantization
nanoNAS.apply_uint8_post_training_quantization()

#evaluate post training quantization
nanoNAS.test_keras_model(path_to_test_set)
nanoNAS.test_tflite_model(path_to_test_set)

Found 9605 files belonging to 2 classes.
Using 6724 files for training.
Found 9605 files belonging to 2 classes.
Using 2881 files for validation.
Epoch 1/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.4796 - loss: 0.6897 - val_accuracy: 0.4825 - val_loss: 0.6939
Epoch 2/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.5756 - loss: 0.6505 - val_accuracy: 0.6012 - val_loss: 0.5982
Epoch 3/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.6185 - loss: 0.6332 - val_accuracy: 0.7605 - val_loss: 0.5796
Saved artifact at '/tmp/tmpre2lr5vt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134688277337040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134688277337616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134688277339728: TensorSpec(shape=(), dtype=t

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(




{'k': 1, 'c': 0, 'RAM': 17408, 'Flash': 3120, 'MACC': 67502, 'max_val_acc': np.float64(0.76)}


Found 9605 files belonging to 2 classes.
Using 6724 files for training.
Found 9605 files belonging to 2 classes.
Using 2881 files for validation.
Epoch 1/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - accuracy: 0.5630 - loss: 0.7110 - val_accuracy: 0.6286 - val_loss: 0.6197
Epoch 2/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.7152 - loss: 0.5823 - val_accuracy: 0.7397 - val_loss: 0.5376
Epoch 3/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.7213 - loss: 0.5750 - val_accuracy: 0.7241 - val_loss: 0.5427
Saved artifact at '/tmp/tmphbetj1ho'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134688277091280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134688277091856

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(




{'k': 1, 'c': 1, 'RAM': 17408, 'Flash': 4216, 'MACC': 90004, 'max_val_acc': np.float64(0.74)}


Found 9605 files belonging to 2 classes.
Using 6724 files for training.
Found 9605 files belonging to 2 classes.
Using 2881 files for validation.
Epoch 1/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 19s 42ms/step - accuracy: 0.5670 - loss: 0.7077 - val_accuracy: 0.7709 - val_loss: 0.5831
Epoch 2/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.7128 - loss: 0.5904 - val_accuracy: 0.7754 - val_loss: 0.5350
Epoch 3/3
421/421 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.7071 - loss: 0.5796 - val_accuracy: 0.7619 - val_loss: 0.5300
Saved artifact at '/tmp/tmpsnlqow8n'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134688277337424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134688277338000

...

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(



Tflite model saved in: /content/results/resulting_architecture.tflite

Found 1000 files belonging to 2 classes.


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9225 - loss: 0.2236

Keras model test accuracy: 0.9110000133514404

Found 1000 files belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)



Tflite model test accuracy: 0.91

Tflite model in: /content/results/resulting_architecture.tflite

